<a href="https://colab.research.google.com/github/MasterAI-EAM/Darwin/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MasterAI-EAM/Darwin.git

Cloning into 'Darwin'...
remote: Enumerating objects: 522, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 522 (delta 26), reused 8 (delta 8), pack-reused 487
Receiving objects: 100% (522/522), 19.73 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [ ]:
!pip install -r Darwin/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cpu


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
!gdown https://drive.google.com/uc?id=1LdzCrbDxxUgT6udB87QmTTEZYs5ZN_EE

# if Access denied with the following error: Too many users have viewed or downloaded this file recently.
# you can manually download this zip file from drive and upload to colab manually.

Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1LdzCrbDxxUgT6udB87QmTTEZYs5ZN_EE 



In [ ]:
!unzip darwin-7b.zip

unzip:  cannot find or open darwin-7b.zip, darwin-7b.zip.zip or darwin-7b.zip.ZIP.


In [ ]:
#load the model
# model_path = 'decapoda-research/llama-7b-hf'
model_path = 'darwin-7b'
print("loading model, path:", model_path)
tokenizer = LlamaTokenizer.from_pretrained(model_path)
# if you use GPU, usetorch_dtype=torch.float16
# if you use cpu, use torchtype=torch.float32
if device=="cuda":
  torch_type = torch.float16
else:
  torch_type = torch.float32
model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=False,
    torch_dtype=torch_type,
    device_map="auto"
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


loading model, path: darwin-7b


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""The following is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction}
    ### Input:
    {input}
    ### Response:"""
    else:
        return f"""The following is an instruction that describes a task. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction}
    ### Response:"""

def process_response(response):
    response = response.split('Response: ')[1].split('\n')[0]
    return response

def evaluate(instruction,
       input = None,
       temperature = 0.8,
       top_p = 0.75,
       top_k=40,
       do_sample=True,
       repetition_penalty=1.0,
       max_new_tokens=256,
       **kwargs):
    prompt = generate_prompt(instruction,input)
    # if use gpu, add .to("cuda")
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    generated_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        repetition_penalty=repetition_penalty,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        **kwargs
    )
    response = tokenizer.decode(generated_ids[0])
    response = process_response(response)
    return response

In [ ]:
for instruction in [
    'Write lipophilicity of given SMILES. CC(C)C(NC(=O)CN1C(=O)C(=CN=C1C2CCCCC2)NC(=O)OCc3ccccc3)C(=O)C(F)(F)F',
    'Given compound, write its potential SELFIES. Decalin',
    'What is water solubility expressed as a logarithm in mol/L of given compound in room temperature? Methyl acrylate',
    'Tell me if given composition has glass formation ability. Ni53.5B44C2.5',
    'Is composition metal? InSb2S4Cl'
]:
    print("Instruction:",instruction)
    print('Response:',evaluate(instruction))
    print('------------------')

Instruction: Write lipophilicity of given SMILES. CC(C)C(NC(=O)CN1C(=O)C(=CN=C1C2CCCCC2)NC(=O)OCc3ccccc3)C(=O)C(F)(F)F


KeyboardInterrupt: ignored

In [ ]:
!ls
!rm -rf darwin-7b
!rm -rf darwin-7b.zip
!rm -rf Darwin

Darwin	darwin-7b  darwin-7b.zip  sample_data
